In [1]:
!pip install -r ../requirements.txt

### Добавляем импорты и считываем переменные

In [2]:
import requests
import numpy as np
import os
import glob
import chromadb
import json

%reload_ext dotenv 
%dotenv ../.env

### Создаем переменные и функциии для работы с YandexGPT Embedding Model
+ `embed_doc` - возвращает embedding документа
+ `embed_query` - возвращает embedding запроса

In [3]:
api_key = os.getenv('YAGPT_API_KEY')
folder_id = os.getenv('YA_FOLDER_ID')

doc_uri = f"emb://{folder_id}/text-search-doc/latest"
query_uri = f"emb://{folder_id}/text-search-query/latest"

embed_url = "https://llm.api.cloud.yandex.net:443/foundationModels/v1/textEmbedding"
headers = {"Content-Type": "application/json", 'Authorization': f'Api-Key {api_key}', "x-folder-id": f"{folder_id}"}

def embed_doc(text):
    json = {
        "modelUri": doc_uri,
        "text": text
    }
    req = requests.post(embed_url, json=json, headers=headers)
    return req.text
    
def embed_query(text):
    json = {
        "modelUri": query_uri,
        "text": text
    }
    req = requests.post(embed_url, json=json, headers=headers)
    return req.text

### Считываем данные из документов в папке documents и создаем список документов

In [4]:
doc_files = glob.glob('/home/ashneider/mount/simple_RAG_chatbot/documents/*.txt')
doc_text = []
for file_name in doc_files:
    file_data = open(file_name, "r").read()
    doc_text.append(file_data)

In [5]:
print([doc for doc in doc_text])

['Инструкция по регистрации аккаунта на сервисе MyApp:\n\n### Шаг 1: Переход на веб-сайт MyApp\n1. Откройте любой веб-браузер на вашем компьютере или мобильном устройстве.\n2. Введите адрес MyApp в адресной строке браузера и нажмите Enter, чтобы перейти на сайт.\n\n### Шаг 2: Нахождение кнопки "Регистрация"\n1. После загрузки сайта MyApp, просмотрите домашнюю страницу и найдите кнопку "Регистрация" или "Создать аккаунт". Обычно эта кнопка располагается в верхнем правом углу страницы или на главной странице.\n\n### Шаг 3: Заполнение регистрационной формы\n1. Щелкните по кнопке "Регистрация". Вас перенаправит на страницу с регистрационной формой.\n2. Заполните поля формы, предоставив необходимую информацию, такую как:\n   - Имя пользователя: Уникальное имя, которое будет использоваться для входа на сервис.\n   - Пароль: Безопасный пароль, состоящий из букв, цифр и специальных символов.\n   - Адрес электронной почты: Действующий адрес электронной почты для подтверждения аккаунта.\n   - Др

### Получаем embedding-и данных из документов

In [6]:
doc_embeddings = []
for t in doc_text:
    doc_embeddings.append(json.loads(embed_doc(t))["embedding"])

### Добавляем embeddings в СromaDB

In [10]:
client = chromadb.Client()

collection = client.get_or_create_collection(name="InternalDocs")
collection.add(
    embeddings = doc_embeddings,
    documents = [doc_text[0], doc_text[1]],
    metadatas = [{"source": "register_account"},{"source": "vocation_request"}],
    ids = ["id1", "id2"]
)

In [11]:
#client.delete_collection("InternalDocs")

### Создаем embedding вектор для запроса к БД и используем его для поиска документа

In [12]:
question_embedding = json.loads(embed_query("Как зарегестрировать аккаунт?"))["embedding"]

result = collection.query(
    query_embeddings=question_embedding,
    n_results=1
)

print(result)

{'ids': [['id1']], 'distances': [[1.1652776002883911]], 'metadatas': [[{'source': 'register_account'}]], 'embeddings': None, 'documents': [['Инструкция по регистрации аккаунта на сервисе MyApp:\n\n### Шаг 1: Переход на веб-сайт MyApp\n1. Откройте любой веб-браузер на вашем компьютере или мобильном устройстве.\n2. Введите адрес MyApp в адресной строке браузера и нажмите Enter, чтобы перейти на сайт.\n\n### Шаг 2: Нахождение кнопки "Регистрация"\n1. После загрузки сайта MyApp, просмотрите домашнюю страницу и найдите кнопку "Регистрация" или "Создать аккаунт". Обычно эта кнопка располагается в верхнем правом углу страницы или на главной странице.\n\n### Шаг 3: Заполнение регистрационной формы\n1. Щелкните по кнопке "Регистрация". Вас перенаправит на страницу с регистрационной формой.\n2. Заполните поля формы, предоставив необходимую информацию, такую как:\n   - Имя пользователя: Уникальное имя, которое будет использоваться для входа на сервис.\n   - Пароль: Безопасный пароль, состоящий из